In [1]:
# Импорт библиотек
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors

import time
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

**Теория:**

Базовый алгоритм совместной фильтрации.

$$\hat{r}_{ui} = \frac{ \sum\limits_{v \in N^k_i(u)} \text{sim}(u, v) \cdot r_{vi}} {\sum\limits_{v \in N^k_i(u)} \text{sim}(u, v)}$$
или
$$\hat{r}_{ui} = \frac{ \sum\limits_{j \in N^k_u(i)} \text{sim}(i, j) \cdot r_{uj}} {\sum\limits_{j \in N^k_u(i)} \text{sim}(i, j)}$$

Однако, в работе нужно использовать базовый алгоритм совместной фильтрации, учитывающий средние оценки каждого пользователя.

$$
\hat{r}_{ui} = \mu_u + \frac{ \sum\limits_{v \in N^k_i(u)} \text{sim}(u, v) \cdot (r_{vi} - \mu_v)} {\sum\limits_{v \in N^k_i(u)} \text{sim}(u, v)}
$$
или
$$
\hat{r}_{ui} = \mu_i + \frac{ \sum\limits_{j \in N^k_u(i)} \text{sim}(i, j) \cdot (r_{uj} - \mu_j)} {\sum\limits_{j \in N^k_u(i)} \text{sim}(i, j)}
$$


Но в итоге можно задать `use_mean_adjustment = (False/True)` для выбора алгоритма.

In [4]:
def create_cf_matrix(train, cf_type='user'):
    """
    Создает матрицу для коллаборативной фильтрации.
    
    Параметры:
    - train: DataFrame с обучающими данными.
    - cf_type: 'user' для пользователь-фильм матрицы или 'item' для фильм-пользователь матрицы.
    
    Возвращает:
    - Матрица для коллаборативной фильтрации.
    """
    if cf_type == 'user':
        # Матрица пользователь-фильм
        return train.pivot(index='userId', columns='movieId', values='rating').fillna(0)
    elif cf_type == 'item':
        # Матрица фильм-пользователь
        return train.pivot(index='movieId', columns='userId', values='rating').fillna(0)
    else:
        raise ValueError("cf_type должен быть 'user' или 'item'")

# 1. Загрузка и разделение данных
def load_and_split_data(file_path, test_size=0.3, random_state=42):
    df = pd.read_csv(file_path)
    train, test = train_test_split(df, test_size=test_size, random_state=random_state)
    return train, test

# 2-3. User-based and Item-based коллаборативная фильтрация с/без z-score
def collaborative_filtering(train, test, cf_type='user', use_mean_adjustment=False, use_z_score=False, N=None):
    # Используем отдельную функцию для создания матрицы
    matrix = create_cf_matrix(train, cf_type=cf_type)
    
    # Вычисляем косинусное сходство
    similarity = cosine_similarity(matrix)
    
    # Вычисляем средние и стандартные отклонения для Z-score (если нужно)
    means = matrix.mean(axis=1)
    stds = matrix.std(axis=1) if use_z_score else None
    
    # Если включен Z-score, нормализуем рейтинги
    if use_z_score:
        matrix_normalized = matrix.sub(means, axis=0).div(stds, axis=0)
    
    # Функция для предсказания рейтинга
    def predict_rating(user_id, item_id):
        if cf_type == 'user':
            # Если тип фильтрации пользователь-фильм
            user_index, item_index = user_id, item_id
        else:
            # Если тип фильтрации фильм-пользователь
            user_index, item_index = item_id, user_id
        
        # Если пользователь или элемент отсутствует в базе
        if user_index not in matrix.index or item_index not in matrix.columns:
            return matrix.mean().mean()
        
        # Рейтинги текущего пользователя или фильма
        ratings = matrix.loc[user_index]
        # Сходства с другими пользователями или фильмами
        similarities = similarity[matrix.index.get_loc(user_index)]
        
        # Сортируем по схожести и выбираем топ-N (если задано)
        sorted_similarities = np.argsort(similarities)[::-1]
        top_similar = sorted_similarities[:N] if N else sorted_similarities
        
        numerator = 0
        denominator = 0
        for similar in top_similar:
            # Проверяем, есть ли рейтинг для указанного элемента (пользователя или фильма)
            if matrix.iloc[similar][item_index] > 0:
                if use_z_score:
                    # Коррекция на Z-score
                    adjusted_rating = matrix_normalized.iloc[similar][item_index]
                    numerator += similarities[similar] * adjusted_rating
                elif use_mean_adjustment:
                    # Коррекция на среднюю оценку
                    adjusted_rating = matrix.iloc[similar][item_index] - means.iloc[similar]
                    numerator += similarities[similar] * adjusted_rating
                else:
                    numerator += similarities[similar] * matrix.iloc[similar][item_index]
                
                # Знаменатель: \text{denominator} += \text{sim}(u, v)
                denominator += similarities[similar]
        
        # Если нет похожих пользователей/фильмов, предсказываем средний рейтинг пользователя
        if denominator == 0:
            return ratings.mean()
        
        if use_z_score:
            # Предсказание через Z-score
            predicted_z_score = numerator / denominator
            return means.loc[user_index] + predicted_z_score * stds.loc[user_index]
        elif use_mean_adjustment:
            # Предсказание с коррекцией на среднее значение
            return means.loc[user_index] + numerator / denominator
        else:
            # Предсказание без коррекции на среднее значение
            return numerator / denominator
    
    # Предсказываем рейтинги для тестового набора
    test['predicted_rating'] = test.apply(lambda row: predict_rating(row['userId'], row['movieId']), axis=1)
    
    # Вычисляем RMSE
    rmse = np.sqrt(mean_squared_error(test['rating'], test['predicted_rating']))
    return rmse


# 2. User-based коллаборативная фильтрация
# def user_based_cf(train, test, use_mean_adjustment=False, N=None):
    # Создаем матрицу пользователь-фильм
    user_item_matrix = train.pivot(index='userId', columns='movieId', values='rating').fillna(0)
    
    # Вычисляем косинусное сходство между пользователями
    user_similarity = cosine_similarity(user_item_matrix)
    
    # Вычисляем средний рейтинг для каждого пользователя (если нужно)
    user_means = user_item_matrix.mean(axis=1) if use_mean_adjustment else None
    
    # Функция для предсказания рейтинга
    def predict_rating(user_id, item_id):
        # Если юзер или фильм отсутствуют в базе, то выдаем среднее значение
        if user_id not in user_item_matrix.index or item_id not in user_item_matrix.columns:
            return user_item_matrix.mean().mean()
        
        # Получаем рейтинги текущего пользователя
        user_ratings = user_item_matrix.loc[user_id]
        # Вычисляем косинусное сходство между текущим пользователем и всеми остальными пользователями
        similar_users = user_similarity[user_item_matrix.index.get_loc(user_id)]
        
        # Сортируем пользователей по схожести
        sorted_users = np.argsort(similar_users)[::-1]
        
        # Если N задан, выбираем только N ближайших пользователей
        top_similar_users = sorted_users[:N] if N else sorted_users
        
        # Инициализируем числитель и знаменатель для формулы
        numerator = 0
        denominator = 0
        for similar_user in top_similar_users:
            # Проверяем, оценил ли пользователь фильм item_id
            if user_item_matrix.iloc[similar_user][item_id] > 0:
                # Если выбран алгоритм с учетом средних оценок
                if use_mean_adjustment:
                    # [r_{vi} - \mu_v] — отклонение рейтинга пользователя v от его среднего
                    adjusted_rating = user_item_matrix.iloc[similar_user][item_id] - user_means.iloc[similar_user]
                    # Числитель: sim(u, v) * (r_{vi} - \mu_v)
                    numerator += similar_users[similar_user] * adjusted_rating
                else:
                    # Числитель: sim(u, v) * r_{vi}
                    numerator += similar_users[similar_user] * user_item_matrix.iloc[similar_user][item_id]
                
                # Знаменатель: \sum sim(u, v)
                denominator += similar_users[similar_user]
        
        # Если знаменатель равен 0, возвращаем средний рейтинг пользователя
        if denominator == 0:
            return user_ratings.mean()
        
        # Если выбран алгоритм с учетом средних оценок
        if use_mean_adjustment:
            # Финальная формула с учетом средних оценок:
            # \hat{r}_{ui} = \mu_u + \frac{\sum sim(u, v) \cdot (r_{vi} - \mu_v)}{\sum sim(u, v)}
            return user_means.loc[user_id] + numerator / denominator
        else:
            # Финальная формула без учета средних оценок:
            # \hat{r}_{ui} = \frac{\sum sim(u, v) \cdot r_{vi}}{\sum sim(u, v)}
            return numerator / denominator
    
    # Предсказываем рейтинги для тестового набора
    test['predicted_rating'] = test.apply(lambda row: predict_rating(row['userId'], row['movieId']), axis=1)
    
    # Вычисляем RMSE
    rmse = np.sqrt(mean_squared_error(test['rating'], test['predicted_rating']))
    return rmse

# 3. Item-based коллаборативная фильтрация
# def item_based_cf(train, test, use_mean_adjustment=False):
    # Создаем матрицу фильм-пользователь
    item_user_matrix = train.pivot(index='movieId', columns='userId', values='rating').fillna(0)
    
    # Вычисляем косинусное сходство между фильмами
    item_similarity = cosine_similarity(item_user_matrix)
    
    # Вычисляем средний рейтинг для каждого фильма (если нужно)
    item_means = item_user_matrix.mean(axis=1) if use_mean_adjustment else None
    
    # Функция для предсказания рейтинга
    def predict_rating(user_id, item_id):
        if item_id not in item_user_matrix.index or user_id not in item_user_matrix.columns:
            return item_user_matrix.mean().mean()
        
        item_ratings = item_user_matrix.loc[item_id]
        similar_items = item_similarity[item_user_matrix.index.get_loc(item_id)]
        
        # Находим топ-10 похожих фильмов
        top_similar_items = np.argsort(similar_items)[-40:-1]
        
        # Вычисляем взвешенный рейтинг
        numerator = 0
        denominator = 0
        for similar_item in top_similar_items:
            if item_user_matrix.iloc[similar_item][user_id] > 0:
                if use_mean_adjustment:
                    # Коррекция рейтинга на среднее значение
                    adjusted_rating = item_user_matrix.iloc[similar_item][user_id] - item_means.iloc[similar_item]
                    numerator += similar_items[similar_item] * adjusted_rating
                else:
                    numerator += similar_items[similar_item] * item_user_matrix.iloc[similar_item][user_id]
                denominator += similar_items[similar_item]
        
        if denominator == 0:
            return item_ratings.mean()
        
        if use_mean_adjustment:
            # Предсказание с корректировкой на среднее значение фильма
            return item_means.loc[item_id] + numerator / denominator
        else:
            return numerator / denominator
    
    # Предсказываем рейтинги для тестового набора
    test['predicted_rating'] = test.apply(lambda row: predict_rating(row['userId'], row['movieId']), axis=1)
    
    # Вычисляем RMSE
    rmse = np.sqrt(mean_squared_error(test['rating'], test['predicted_rating']))
    return rmse

# 4-5. Рекомендательная система на основе кластеризации и расчета средней оценки кластера
def cluster_based_cf(train, test, n_clusters=40, use_z_score=False, cf_type='user'):
    """
    Кластерная коллаборативная фильтрация.
    
    Параметры:
    - train: DataFrame с обучающими данными.
    - test: DataFrame с тестовыми данными.
    - n_clusters: количество кластеров для K-means.
    - use_z_score: флаг для использования Z-score нормализации.
    - cf_type: 'user' для пользователь-фильм матрицы или 'item' для фильм-пользователь матрицы.
    
    Возвращает:
    - RMSE для предсказанных значений.
    """
    # Создаем матрицу с помощью create_cf_matrix
    matrix = create_cf_matrix(train, cf_type=cf_type)
    
    if use_z_score:
        # Нормализуем оценки с помощью z-score
        means = matrix.mean(axis=1)
        stds = matrix.std(axis=1)
        matrix_normalized = matrix.sub(means, axis=0).div(stds, axis=0)
        matrix_for_clustering = matrix_normalized
    else:
        # Используем обычную матрицу для кластеризации
        matrix_for_clustering = matrix
    
    # Применяем K-means кластеризацию
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(matrix_for_clustering)
    
    # Функция для предсказания рейтинга
    def predict_rating(user_id, item_id):
        if cf_type == 'user':
            user_index, item_index = user_id, item_id
        else:
            user_index, item_index = item_id, user_id
        
        if user_index not in matrix.index or item_index not in matrix.columns:
            return matrix.mean().mean()
        
        # Определяем кластер пользователя или фильма
        cluster = clusters[matrix.index.get_loc(user_index)]
        cluster_items = matrix.index[clusters == cluster]
        
        if use_z_score:
            # Получаем нормализованные рейтинги для кластера
            cluster_ratings_normalized = matrix_normalized.loc[cluster_items, item_index]
            if cluster_ratings_normalized.empty:
                return means[user_index]
            predicted_z_score = cluster_ratings_normalized.mean()
            # Преобразуем предсказание обратно в исходные единицы
            return means[user_index] + (predicted_z_score * stds[user_index])
        else:
            # Получаем обычные рейтинги для кластера
            cluster_ratings = matrix.loc[cluster_items, item_index]
            if cluster_ratings.empty:
                return matrix.mean().mean()
            return cluster_ratings.mean()
    
    # Предсказываем рейтинги для тестового набора
    test['predicted_rating'] = test.apply(lambda row: predict_rating(row['userId'], row['movieId']), axis=1)
    
    # Вычисляем RMSE
    rmse = np.sqrt(mean_squared_error(test['rating'], test['predicted_rating']))
    return rmse

# 5. Рекомендательная система на основе кластеризации и учета z-оценки
# def cluster_based_cf_z_score(train, test, n_clusters=40):
    # Создаем матрицу пользователь-фильм
    user_item_matrix = train.pivot(index='userId', columns='movieId', values='rating').fillna(0)
    
    # Нормализуем оценки с помощью z-score
    user_means = user_item_matrix.mean(axis=1)
    user_stds = user_item_matrix.std(axis=1)
    user_item_matrix_normalized = user_item_matrix.sub(user_means, axis=0).div(user_stds, axis=0)
    
    # Применяем K-means кластеризацию
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    user_clusters = kmeans.fit_predict(user_item_matrix_normalized)
    
    # Функция для предсказания рейтинга
    def predict_rating(user_id, item_id):
        if user_id not in user_item_matrix.index or item_id not in user_item_matrix.columns:
            return user_item_matrix.mean().mean()
        
        user_cluster = user_clusters[user_item_matrix.index.get_loc(user_id)]
        cluster_users = user_item_matrix.index[user_clusters == user_cluster]
        cluster_ratings_normalized = user_item_matrix_normalized.loc[cluster_users, item_id]
        
        if cluster_ratings_normalized.empty:
            return user_means[user_id]
        
        predicted_z_score = cluster_ratings_normalized.mean()
        return user_means[user_id] + (predicted_z_score * user_stds[user_id])
    
    # Предсказываем рейтинги для тестового набора
    test['predicted_rating'] = test.apply(lambda row: predict_rating(row['userId'], row['movieId']), axis=1)
    
    # Вычисляем RMSE
    rmse = np.sqrt(mean_squared_error(test['rating'], test['predicted_rating']))
    return rmse

# 6. Рекомендательная система на основе голосования
def voting_based_cf(train, test, cf_type='user', n_neighbors=40, use_knn=False):
    """
    Коллаборативная фильтрация с голосованием на основе сходства.

    Параметры:
    - train: DataFrame с обучающими данными.
    - test: DataFrame с тестовыми данными.
    - cf_type: 'user' для пользователь-фильм матрицы или 'item' для фильм-пользователь матрицы.
    - n_neighbors: количество соседей для KNN или топ-N для голосования.
    - use_knn: флаг для использования KNN вместо простого выбора топ-N похожих пользователей/фильмов.

    Возвращает:
    - RMSE для предсказанных значений.
    """
    # Создаем матрицу для коллаборативной фильтрации
    matrix = create_cf_matrix(train, cf_type=cf_type)
    
    # Вычисляем сходство
    similarity = cosine_similarity(matrix)

    # Опционально применяем KNN для поиска соседей
    if use_knn:
        knn = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine').fit(matrix)
    
    # Функция для предсказания рейтинга
    def predict_rating(user_id, item_id):
        if cf_type == 'user':
            user_index, item_index = user_id, item_id
        else:
            user_index, item_index = item_id, user_id
        
        if user_index not in matrix.index or item_index not in matrix.columns:
            return matrix.mean().mean()

        # Если используем KNN
        if use_knn:
            distances, neighbors = knn.kneighbors([matrix.loc[user_index]], n_neighbors=n_neighbors)
            neighbors = neighbors.flatten()
        else:
            # Поиск топ-N похожих пользователей/фильмов
            similarities = similarity[matrix.index.get_loc(user_index)]
            neighbors = np.argsort(similarities)[-n_neighbors-1:-1]
        
        # Собираем голоса (рейтинги соседей)
        neighbor_ratings = matrix.iloc[neighbors][item_index].values
        valid_ratings = neighbor_ratings[neighbor_ratings > 0]

        # Если нет валидных рейтингов, возвращаем средний рейтинг
        if len(valid_ratings) == 0:
            return matrix.loc[user_index].mean()

        # Возвращаем среднее значение рейтингов соседей
        return valid_ratings.mean()

    # Предсказываем рейтинги для тестового набора
    test['predicted_rating'] = test.apply(lambda row: predict_rating(row['userId'], row['movieId']), axis=1)
    
    # Вычисляем RMSE
    rmse = np.sqrt(mean_squared_error(test['rating'], test['predicted_rating']))
    return rmse

# Оформление результата
def create_results_dataframe(results):
    """
    Создает pandas DataFrame из словаря результатов.
    
    :param results: Словарь с названиями методов в качестве ключей и RMSE в качестве значений
    :return: pandas DataFrame с результатами
    """
    df = pd.DataFrame(results).T
    df.columns = ['RMSE', 'Время (с)']
    df.index.name = 'Метод'
    return df

# Функция для замера времени
def time_function(func, *args, **kwargs):
    start_time = time.time()
    result = func(*args, **kwargs)
    elapsed_time = time.time() - start_time
    return result, elapsed_time

# Основная функция
def main():
    # Загрузка и разделение данных
    train, test = load_and_split_data('data/ml-latest-small/ratings.csv', test_size=0.05)
    
    results = {}

    # User-based CF
    (rmse, elapsed_time) = time_function(collaborative_filtering, train, test, cf_type='user', N=100)
    results['user_based'] = [rmse, elapsed_time]
    logger.info(f'user_based ✔ (время: {elapsed_time:.2f} с)')

    # User-based CF with Mean
    (rmse, elapsed_time) = time_function(collaborative_filtering, train, test, cf_type='user', use_mean_adjustment=True, N=100)
    results['user_based_mean'] = [rmse, elapsed_time]
    logger.info(f'user_based_mean ✔ (время: {elapsed_time:.2f} с)')

    # User-based CF with z-score
    (rmse, elapsed_time) = time_function(collaborative_filtering, train, test, cf_type='user', use_z_score=True, N=100)
    results['user_based_z-score'] = [rmse, elapsed_time]
    logger.info(f'user_based_z-score ✔ (время: {elapsed_time:.2f} с)')

    # Item-based CF
    (rmse, elapsed_time) = time_function(collaborative_filtering, train, test, cf_type='item', N=100)
    results['item_based'] = [rmse, elapsed_time]
    logger.info(f'item_based ✔ (время: {elapsed_time:.2f} с)')

    # Item-based CF with Mean
    (rmse, elapsed_time) = time_function(collaborative_filtering, train, test, cf_type='item', use_mean_adjustment=True, N=100)
    results['item_based_mean'] = [rmse, elapsed_time]
    logger.info(f'item_based_mean ✔ (время: {elapsed_time:.2f} с)')

    # Item-based CF with z-score
    (rmse, elapsed_time) = time_function(collaborative_filtering, train, test, cf_type='item', use_z_score=True, N=100)
    results['item_based_z-score'] = [rmse, elapsed_time]
    logger.info(f'item_based_z-score ✔ (время: {elapsed_time:.2f} с)')

    # Cluster-based CF
    (rmse, elapsed_time) = time_function(cluster_based_cf, train, test, n_clusters=5, use_z_score=False, cf_type='user')
    results['user_cluster_based'] = [rmse, elapsed_time]
    logger.info(f'user_cluster_based ✔ (время: {elapsed_time:.2f} с)')

    # Cluster-based CF with z-score
    (rmse, elapsed_time) = time_function(cluster_based_cf, train, test, n_clusters=5, use_z_score=True, cf_type='user')
    results['user_cluster_based_z-score'] = [rmse, elapsed_time]
    logger.info(f'user_cluster_based_z-score ✔ (время: {elapsed_time:.2f} с)')

    # Cluster-based CF
    (rmse, elapsed_time) = time_function(cluster_based_cf, train, test, n_clusters=5, use_z_score=False, cf_type='item')
    results['item_cluster_based'] = [rmse, elapsed_time]
    logger.info(f'item_cluster_based ✔ (время: {elapsed_time:.2f} с)')

    # Cluster-based CF with z-score
    (rmse, elapsed_time) = time_function(cluster_based_cf, train, test, n_clusters=5, use_z_score=True, cf_type='item')
    results['item_cluster_based_z-score'] = [rmse, elapsed_time]
    logger.info(f'item_cluster_based_z-score ✔ (время: {elapsed_time:.2f} с)')

    # Voting-based CF
    (rmse, elapsed_time) = time_function(voting_based_cf, train, test, cf_type='user', n_neighbors=40, use_knn=False)
    results['user_voting_based'] = [rmse, elapsed_time]
    logger.info(f'user_voting_based ✔ (время: {elapsed_time:.2f} с)')

    # Voting-based CF
    (rmse, elapsed_time) = time_function(voting_based_cf, train, test, cf_type='user', n_neighbors=40, use_knn=True)
    results['user_voting_based_knn'] = [rmse, elapsed_time]
    logger.info(f'user_voting_based_knn ✔ (время: {elapsed_time:.2f} с)')

    # Voting-based CF
    (rmse, elapsed_time) = time_function(voting_based_cf, train, test, cf_type='item', n_neighbors=40, use_knn=False)
    results['item_voting_based'] = [rmse, elapsed_time]
    logger.info(f'item_voting_based ✔ (время: {elapsed_time:.2f} с)')

    # Voting-based CF
    (rmse, elapsed_time) = time_function(voting_based_cf, train, test, cf_type='item', n_neighbors=40, use_knn=True)
    results['item_voting_based_knn'] = [rmse, elapsed_time]
    logger.info(f'item_voting_based_knn ✔ (время: {elapsed_time:.2f} с)')

    # Создаем DataFrame с результатами
    results_df = create_results_dataframe(results)
    
    # Выводим результаты
    logger.info(f'\n{results_df.sort_values("RMSE")}')


if __name__ == "__main__":
    main()

INFO:__main__:user_based ✔ (время: 25.76 с)
INFO:__main__:user_based_mean ✔ (время: 25.48 с)
INFO:__main__:user_based_z-score ✔ (время: 24.80 с)
INFO:__main__:item_based ✔ (время: 28.86 с)
INFO:__main__:item_based_mean ✔ (время: 29.63 с)
INFO:__main__:item_based_z-score ✔ (время: 27.72 с)
INFO:__main__:user_cluster_based ✔ (время: 5.27 с)
INFO:__main__:user_cluster_based_z-score ✔ (время: 5.44 с)
INFO:__main__:item_cluster_based ✔ (время: 6.02 с)
INFO:__main__:item_cluster_based_z-score ✔ (время: 6.56 с)
INFO:__main__:user_voting_based ✔ (время: 10.16 с)
INFO:__main__:user_voting_based_knn ✔ (время: 238.29 с)
INFO:__main__:item_voting_based ✔ (время: 7.48 с)
INFO:__main__:item_voting_based_knn ✔ (время: 238.53 с)
INFO:__main__:
                                RMSE   Время (с)
Метод                                           
user_based                  1.210142   25.756976
user_based_mean             1.225115   25.476962
user_voting_based           1.319137   10.157130
user_voting_based

Признаю, что данные не 70/30 в результате, но это из-за того, что модели могут очень долго отрабатывать.